<h1>Hidden Markov Model</h1>
<p> For this project we will be constructing a hidden markov model and with that will be predicting text and also generating text as well </p>
<p>Imports</p>

In [97]:
import numpy as np
import pandas as pd
from itertools import chain
import random
from tqdm import tqdm

<h3>Utility functions</h3>

In [98]:
def importData(filename):
    allLines = []
    with open(filename, 'r') as f:
        for line in tqdm(f):
            clean_line = line.lower()
            remove_chars = ['\n', '\t', '"','-']
            punc_chars = [';','.',',','!','?','(',')',':','|','[',']',"'"]
            for char in remove_chars:
                clean_line = clean_line.replace(char, ' ')
            for char in punc_chars:
                clean_line = clean_line.replace(char, '')
            split_sentence = clean_line.split()
            if split_sentence and (split_sentence[0] == "act" or split_sentence[0] == "scene"):
                continue
            allLines.append(split_sentence)
        uniqLinesLen = len(sorted(set(chain.from_iterable(allLines))))
        return allLines, uniqLinesLen
    
def cleanSentence(sample_text):
    sample_text = sample_text.lower()
    remove_chars = ['\n', '\t', '"','-']
    punc_chars = [';','.',',','!','?','(',')',':','|','[',']',"'"]
    for char in remove_chars:
        sample_text = sample_text.replace(char, ' ')
    for char in punc_chars:
        sample_text = sample_text.replace(char, '')
    return sample_text.split()

def createDict(tokens, emiss_data):
    ret_dict = {}
    for k, v in zip(tokens, emiss_data):
        ret_dict[k] = v
    return ret_dict

def getProbDict(wordsList):
    prob_dict = {}
    for word in wordsList:
        prob_dict[word] = prob_dict.get(word, 0) + 1
    return {k:prob_dict[k]/len(wordsList) for k in prob_dict}

Clean Data

In [99]:
cleanedText, numUnique = importData('../data/alllines.txt')
cleanedText

111396it [00:00, 174091.05it/s]


[['enter',
  'king',
  'henry',
  'lord',
  'john',
  'of',
  'lancaster',
  'the',
  'earl',
  'of',
  'westmoreland',
  'sir',
  'walter',
  'blunt',
  'and',
  'others'],
 ['so', 'shaken', 'as', 'we', 'are', 'so', 'wan', 'with', 'care'],
 ['find', 'we', 'a', 'time', 'for', 'frighted', 'peace', 'to', 'pant'],
 ['and', 'breathe', 'short', 'winded', 'accents', 'of', 'new', 'broils'],
 ['to', 'be', 'commenced', 'in', 'strands', 'afar', 'remote'],
 ['no', 'more', 'the', 'thirsty', 'entrance', 'of', 'this', 'soil'],
 ['shall', 'daub', 'her', 'lips', 'with', 'her', 'own', 'childrens', 'blood'],
 ['nor', 'more', 'shall', 'trenching', 'war', 'channel', 'her', 'fields'],
 ['nor', 'bruise', 'her', 'flowerets', 'with', 'the', 'armed', 'hoofs'],
 ['of', 'hostile', 'paces', 'those', 'opposed', 'eyes'],
 ['which', 'like', 'the', 'meteors', 'of', 'a', 'troubled', 'heaven'],
 ['all', 'of', 'one', 'nature', 'of', 'one', 'substance', 'bred'],
 ['did', 'lately', 'meet', 'in', 'the', 'intestine', 'shock

<h3>Hidden Markov Model</h3>
<h4>Background</h4>
<p>For this model, it is a purely statistical model that assumes that it is modeling a markov process with certain undiscernable states within the system. Essentially, the model determines the next state based off of the current or previous state. 
</p>
<h4>Implementation</h4>
<p>For this project we are assuming that the states we are observing are words in a sentence. We will be using both the previouos and second to previous word as the previous states to the prediction of the next state. For this we will be creating state matrices and a transition matrix when transition between states or words in a sentence. As the normal strucuture of a sentence we have starting and ending states. The starting and ending points will be indicated so that the generated sentences will have grounding in the starting and ending of the phrase.
</p>

The steps for completing this project are as follows:
<ul>
    <li/> Importing and cleaning data
    <li/> Creating the state and transition matrices
    <li/> Creating the functions to then use the calculated probabilities for generating and predicting text
</ul>

In [159]:
class HMM():
    def __init__(self):
        #we can use this because we know there are no caps in the dataset
        self.startWord = {}
        self.secWord = {}
        self.trans = {}
    
    def fit(self, X):
        self.create_transitions_from_corpus(X)
        self.calculate_prob_distributions()
        
                
    def create_transitions_from_corpus(self, corpus):
        uniqueWords = set()
        for line in corpus:
            for i in range(len(line)):
                if i == 0:
                    #initial state
                    #if we get a 1 then we know that there is nothing in the start word dict
                    self.startWord[line[i]] = self.startWord.get(line[i], 0) + 1
                elif i == 1:
                    #second state
                    if line[i-1] not in self.secWord:
                        self.secWord[line[i-1]] = []
                    self.secWord[line[i-1]].append(line[i])
                elif i == len(line) - 1:
                    #stop state
                    if (line[i-1], line[i]) not in self.trans:
                        self.trans[(line[i-1], line[i])] = []
                    self.trans[(line[i-1], line[i])].append("STOP")
                else:
                    #everything inbetween
                    if (line[i-2], line[i-1]) not in self.trans:
                        self.trans[(line[i-2], line[i-1])] = []
                    self.trans[(line[i-2], line[i-1])].append(line[i])
                    
    def calculate_prob_distributions(self):
        first_freq_count = 0
        for key in self.startWord:
            first_freq_count += self.startWord[key]
        for key in self.startWord:
            self.startWord[key] = self.startWord[key]/first_freq_count
            
        for key in self.secWord:
            self.secWord[key] = getProbDict(self.secWord[key])
        for key in self.trans:
            self.trans[key] = getProbDict(self.trans[key])
            
    def getRandWord(self):
        return random.choices(list(self.startWord.keys()),weights=list(self.startWord.values()))
    
    def getWord(self, beginWord):
        return random.choices(list(self.secWord[beginWord].keys()), weights=list(self.secWord[beginWord].values()))
    
    def getNextWord(self, firWord, secWord):
        
        reduced_dict = {k:self.trans[(firWord, secWord)][k] for k in self.trans[(firWord, secWord)] if 
                        self.trans[(firWord, secWord)][k] != secWord}
        return random.choices(list(reduced_dict.keys()),weights=list(reduced_dict.values()))

    def text_prediction(self, corpus_text):
        #needs at least two words to work and then creates a full sentence from that
        sample_text = cleanSentence(corpus_text)
        if len(sample_text) < 2:
            print("Please provide at least 2 words to start with")
            return
        zeroWord = sample_text[0]
        firWord = sample_text[1]
        secLastWord = ''
        while secLastWord != "STOP":
            try:
                secWord = self.getNextWord(zeroWord, firWord)[0]
                sample_text.append(secWord)
                zeroWord = firWord
                firWord = secWord
            except Exception as e:
#                 print(e)
                break
        if sample_text[-1] == "STOP":
            return(' '.join(sample_text[:-1]))
        else:
            return(' '.join(sample_text))
        
    def generate_text(self):
        #generates a random sentence. One sentence was just done for ease
        sample_text = []
        zeroWord = self.getRandWord()
        firWord = self.getWord(zeroWord)
        secWord = ""
        sample_text.append(zeroWord)
        sample_text.append(firWord)
        while secWord != "STOP":
            try:
                secWord = self.getNextWord(zeroWord, firWord)[0]
                sample_text.append(secWord)
                zeroWord = firWord
                firWord = secWord
            except Exception as e:
#                 print(e)
                break
        if sample_text[-1] == "STOP":
            return(' '.join(sample_text[:-1]))
        else:
            return(' '.join(sample_text))

In [160]:
model = HMM()
model.fit(cleanedText)

In [28]:
model.generate_text()

'answer my good fellows will bring the keys there sits a king my fathers signet in my breath'

In [162]:
model.text_prediction("how might i")

'how might i we but learn from whence i think i love her i am combined by a pace goes backward with a thought of added honour torn from forth the cruel issue of the ladys which is that says such a heady currance scouring'

In [168]:
model.text_prediction("king and")

'king and did seat the throne'

<h3>Conclusion</h3>
<p> From this, we are able to create text with the hint of a shakespearean twist. One note should be that I reduce the lines of text by breaking if the words are not within the transition states to produce the next word, which leaves shorter sentences but still seem to have some meaning. Anothing thing to note is that while the conjoined words seem to have some relative meaning, as a whole it seems to provide very little in terms of substance. One possible explination for this would be how little information this type of model is able to hold. It maintains state of only the previous two states. The two states do still maintain a chain like effect in maintaining a meaningful information</p>